In [1]:
%matplotlib inline

import warnings
import time
import sys
import numpy as np
import matplotlib.pyplot as plt
from hmmlearn import hmm

import random
import pandas as pd

pd.options.mode.use_inf_as_na = True
# warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
NUM_TEST = 100
K = 50
NUM_ITERS=10000
DEFAULT_TOL = 1e-12

In [59]:
likelihood_vect = np.empty([0,1])
aic_vect = np.empty([0,1])
bic_vect = np.empty([0,1])

# Possible number of states in Markov Model
STATE_SPACE = range(2,19,1)

In [48]:

# dataset = np.genfromtxt('../Dataset/' + stock + '.csv', delimiter=',')
csv_data = pd.read_csv('BABA.csv')

In [49]:
# clean up data
# delete extra data (Date[0],Adj Close[4], Volume[5]) and leave Open/Hight/Low/Close
del(csv_data['Date'])
del(csv_data['Adj Close'])
# del(csv_data['Volume'])

csv_data.head()

,Open,High,Low,Close,Volume
0,92.699997,99.699997,89.949997,93.889999,271879400
1,92.699997,92.949997,89.500000,89.889999,66657800
2,88.940002,90.480003,86.620003,87.169998,39009800
3,88.470001,90.570000,87.220001,90.570000,32088000
4,91.089996,91.500000,88.500000,88.919998,28598000


In [50]:
csv_data['Volume'] = (csv_data['Volume'] / 1000).astype(int) / 100
csv_data.head()

,Open,High,Low,Close,Volume
0,92.699997,99.699997,89.949997,93.889999,2718.79
1,92.699997,92.949997,89.500000,89.889999,666.57
2,88.940002,90.480003,86.620003,87.169998,390.09
3,88.470001,90.570000,87.220001,90.570000,320.88
4,91.089996,91.500000,88.500000,88.919998,285.98


In [51]:
def get_exp_preprocessing(df,alpha=0.9):
    edata = df.ewm(alpha=alpha).mean()
    return edata

In [52]:
dataset = get_exp_preprocessing(csv_data)
dataset.head()

,Open,High,Low,Close,Volume
0,92.699997,99.699997,89.949997,93.889999,2718.790000
1,92.699997,93.563633,89.540909,90.253635,853.135455
2,89.312614,90.785588,86.909462,87.475584,435.977387
3,88.554186,90.591539,87.188975,90.260837,332.379379
4,90.836438,91.409162,88.368909,89.054070,290.619520


In [53]:
# dataset = (dataset*100).astype(int)/100
# dataset[:,-1] = dataset[:,-1] / 1000

In [54]:
predicted_stock_data = np.empty([0,dataset.shape[1]])
likelihood_vect = np.empty([0,1])
aic_vect = np.empty([0,1])
bic_vect = np.empty([0,1])

In [55]:
dataset = np.array(dataset)

In [56]:
# TEST START

In [57]:
# states = 9

# num_params = states**2 + states
# dirichlet_params_states = np.random.randint(1,50,states)
# #model = hmm.GaussianHMM(n_components=states, covariance_type='full', startprob_prior=dirichlet_params_states, transmat_prior=dirichlet_params_states, tol=0.0001, n_iter=NUM_ITERS, init_params='mc')
# model = hmm.GaussianHMM(n_components=states, covariance_type='full', tol=DEFAULT_TOL, n_iter=NUM_ITERS)
# model.fit(dataset[NUM_TEST:,:])
# if model.monitor_.iter == NUM_ITERS:
#     print('Increase number of iterations')
#     sys.exit(1)
# likelihood_vect = np.vstack((likelihood_vect, model.score(dataset)))

In [ ]:
# aic_vect = np.vstack((aic_vect, -2 * model.score(dataset) + 2 * num_params))
# bic_vect = np.vstack((bic_vect, -2 * model.score(dataset) +  num_params * np.log(dataset.shape[0])))

In [ ]:
# bic_vect

In [ ]:
# TEST END

In [60]:
for states in STATE_SPACE:
    print(states)
    num_params = states**2 + states
    dirichlet_params_states = np.random.randint(1,50,states)
    #model = hmm.GaussianHMM(n_components=states, covariance_type='full', startprob_prior=dirichlet_params_states, transmat_prior=dirichlet_params_states, tol=0.0001, n_iter=NUM_ITERS, init_params='mc')
    model = hmm.GaussianHMM(n_components=states, covariance_type='full', tol=DEFAULT_TOL, n_iter=NUM_ITERS)
    model.fit(dataset[NUM_TEST:,:])
    if model.monitor_.iter == NUM_ITERS:
        print('Increase number of iterations')
        sys.exit(1)
    likelihood_vect = np.vstack((likelihood_vect, model.score(dataset)))
    aic_vect = np.vstack((aic_vect, -2 * model.score(dataset) + 2 * num_params))
    bic_vect = np.vstack((bic_vect, -2 * model.score(dataset) +  num_params * np.log(dataset.shape[0])))

opt_states = np.argmin(bic_vect) + 2
print('Optimum number of states are {}'.format(opt_states))

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
Optimum number of states are 10


In [73]:
df = pd.DataFrame(list(zip(aic_vect,bic_vect)), 
                  columns=("AIC","BIC"),
                 index = np.arange(2,20,1))
df

,AIC,BIC
2,[30452.496953848044],[30482.17303779464]
3,[28766.786473679902],[28826.138641573092]
4,[29072.30899673127],[29171.229276553255]
5,[28787.255988684952],[28935.63640841793]
6,[29051.205433739684],[29258.93802136585]
7,[29149.046697952745],[29426.0234814543]
8,[28292.196045930446],[28648.30905328959]
9,[28194.1080768255],[28639.249336024433]
10,[27909.286422962206],[28453.34796198312]
11,[33909.63621603637],[34562.51006286147]


In [22]:
opt_states = 10